In [7]:
##

In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
prefix = 'mlops/data'

## Test Data

In [7]:
test_x_path = 's3://sagemaker-us-east-1-752847213914/mlops/data/test/test_x.csv'
test_y_path = 's3://sagemaker-us-east-1-752847213914/mlops/data/test/test_y.csv'

In [136]:
test_y = pd.read_csv(test_y_path, header = None)
test_y.columns = ['y']

In [137]:
test_y.shape

(10000, 1)

## Model Prediction

In [21]:
from sagemaker.transformer import Transformer

transformer = Transformer(model_name='rf-scikit-2023-07-14-09-27-43-800',instance_count=1,instance_type='ml.m5.2xlarge')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [23]:
#prediction = transformer.transform(test_x_path)

In [ ]:
sklearn_transformer = sklearn_estimator.transformer(
    model_name='rf-scikit-2023-07-14-09-27-43-800',
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=transformer_output_path
)

In [27]:
# cell 18
transformer_output_path = f"s3://{bucket}/{prefix}/transformer-output"

sklearn_transformer = Transformer(
    model_name='rf-scikit-2023-07-14-09-27-43-800',
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=transformer_output_path
)

sklearn_transformer.transform(
    data=test_x_path,
    data_type='S3Prefix',
    content_type='text/csv'
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating transform job with name: sagemaker-scikit-learn-2023-07-14-12-02-45-181


...........................2023-07-14 12:07:19,903 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-07-14 12:07:19,907 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-07-14 12:07:19,908 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      retur

In [86]:
# cell 19
!aws s3 cp $transformer_output_path/test_x.csv.out /tmp/predictions.csv

download: s3://sagemaker-us-east-1-752847213914/mlops/data/transformer-output/test_x.csv.out to ../../../../tmp/predictions.csv


In [98]:
with open('/tmp/predictions.txt', 'r') as r:
    a = r.read()
prediction = [int(x) for x in a[1:-1].split(',')]
prediction_df = pd.DataFrame({'prediction':prediction})

In [126]:
prediction_df.shape, test_y.shape

((10000, 1), (10000, 1))

In [139]:
# cell 21
pd.crosstab(index=test_y['y'].values, columns=prediction_df['prediction'].values, rownames=['actuals'], colnames=['predictions'])

predictions,0,1,2,3,4,5
actuals,,,,,,
0,1705,0,0,0,0,0
1,0,1579,81,0,0,0
2,0,72,1599,0,0,0
3,0,0,0,1591,11,20
4,0,0,0,5,1660,15
5,0,0,0,11,19,1632


### Data Analytics